Импортируем нужные библиотеки.

In [ ]:
import numpy as np 
import pandas as pd 
import fasttext
from faker import Faker
from sdv.tabular import GaussianCopula

Для генерации фамилий датасета решено использовать библиотеку, которая генерирует синтетические данные. 

Конкретно здесь используются метод last_name, который сгенерирует массив из 100 тысяч фамилий, позже преобразованный в датафрейм.

In [54]:
fake = Faker("en_US")

In [55]:
surnames = []
for _ in range(100000):
    surnames.append(fake.last_name())

In [56]:
df = pd.DataFrame(surnames)
df.head()

,0
0,Frazier
1,Tucker
2,Mcdonald
3,Washington
4,White


Нулевых значений в таблице нет (но если бы были, их нужно было бы убрать).

In [57]:
df=df.dropna(axis=0)

Меняем название столбца для удобства.

In [59]:
df.columns = ['last_name']

In [60]:
df.head()

,last_name
0,Frazier
1,Tucker
2,Mcdonald
3,Washington
4,White


Поскольку в сгенерированном датасете достаточно много повторений, было также принято решение дополнить датасет 
данными, сгенерированными на основе первого датасета. 

Для этого используется библиотека SDV, с помощью которого добавляем в датасет 50 тысяч новых значений.

Соединяем датасеты.

In [61]:
model = GaussianCopula()
model.fit(df)

In [62]:
sample = model.sample(50000)
sample.head()

,last_name
0,Moore
1,Castillo
2,Miller
3,Pierce
4,Greer


In [63]:
df = pd.concat([df,sample])

Посмотрим на дупликаты. Их много...

In [74]:
dups = df.pivot_table(columns=['last_name'], aggfunc='size')
print(dups)

last_name
abbott        61
acevedo       62
acosta        85
adams        569
adkins       108
            ... 
zamora        79
zavala        37
zhang         50
zimmerman     92
zuniga        44
Length: 1000, dtype: int64


Поскольку данные представлены в "чистом" виде, предварительная обработка (с помощью регулярных выражений, например) не нужна. 

Предварительная обработка нужна для предложений, больших текстов.

In [66]:
def preprocess(text):
    #text = re.sub(r'[^\w\s\']',' ', text)
    #text = re.sub(r'[ \n]+', ' ', text)
    #text = re.sub(r'[0-9]+', ' ', text)
    return text.strip().lower() 

In [67]:
df.last_name = df.last_name.map(preprocess)

In [68]:
df.to_csv("surnames.txt", columns=["last_name"], header=None, index=False)

In [ ]:
Самая главная часть – создание модели на основе библиотеки FastText. 

FastText специализируется на создании векторов слов, которые помогают находить семантическую похожесть или
похожесть по составу.

Наиболее важными параметрами модели являются ее размерность и диапазон размеров подслов. Размерность (dim) 
определяет размер векторов, чем они больше, тем больше информации они могут захватить, но для изучения требуется 
больше данных. Но, если они слишком большие, их труднее и медленнее тренировать. По умолчанию мы используем 
100 размеров, но также популярны любые значения от 100 до 300. Подслова — это все подстроки, содержащиеся 
в слове между минимальным размером (minn) и максимальным размером (maxn). По умолчанию мы берем все подслова 
от 3 до 6 символов, но для разных языков может быть более подходящим другой диапазон.

In [87]:
model = fasttext.train_unsupervised("surnames.txt", minn=3, maxn=6, dim=300)

Read 0M words
Number of words:  1001
Number of labels: 0
Progress: 100.0% words/sec/thread: 2086338 lr:  0.000000 avg.loss:  3.230946 ETA:   0h 0m 0s


У FastText есть несколько методов, которые могут найти похожие значения из словаря. 

Например, get_nearest_neighbors показывает похожие слова. Но в данном случае поиск семантически похожих слов не имеет смысла, поскольку мы работаем с фамилиями.

In [88]:
model.get_nearest_neighbors("rich")

[(0.9999931454658508, 'chavez'),
 (0.9999918937683105, 'vasquez'),
 (0.999991774559021, 'velazquez'),
 (0.9999914765357971, 'hull'),
 (0.9999907612800598, 'bird'),
 (0.9999898672103882, 'black'),
 (0.9999896287918091, 'zuniga'),
 (0.9999890923500061, 'scott'),
 (0.9999886155128479, 'wyatt'),
 (0.9999880194664001, 'huffman')]

In [89]:
model.get_nearest_neighbors("ander")

[(0.9999982118606567, 'brennan'),
 (0.9999974370002747, 'rose'),
 (0.9999971389770508, 'swanson'),
 (0.9999971389770508, 'reeves'),
 (0.9999970197677612, 'harrison'),
 (0.9999969005584717, 'lane'),
 (0.9999968409538269, 'hanson'),
 (0.9999968409538269, 'dean'),
 (0.9999967217445374, 'ferrell'),
 (0.9999966621398926, 'lawson')]

Какой итог можно сделать?

Как мне показалось, модель не совсем хорошо справляется с поиском слов, похожих по составу. Возможно, дело не в самом хорошем сгенерированном датасете.